In [11]:
# Libraries, parameters

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import norm
from scipy.integrate import quad
from scipy.interpolate import interp1d, interpn, RegularGridInterpolator
from scipy.signal import fftconvolve
from scipy.stats import poisson
from scipy.optimize import minimize
import math

# Norms for the PMNS matrix squared:
u_e = [0, 0.674743, 0.302844, 0.0224125]
u_m = [0, 0.0946105, 0.360415, 0.544974]
u_t = [0, 0.230646, 0.33674,  0.432613]


# Conversion factor to go from kpc to 1/eV
conv_factor = 3.086e19 * 5.06773093741 * 1e6

# Conversion factor to go from kpc to m
kpctom = 3.086e19
kpctocm = 3.086e22

# More parameters:
phi0 = 100 # TeV^-1 cm^-2 s^-1, completely random, will be able to calculate this directly I think
e0 = 1 # TeV
index = 2.0 # Completely random, not sure how to calculate this exactly
width = 0.1 # In log10 scale, for cascade-like events, this is being not generous

# del_m2
dm2s = (0, 0, 0)

base = 10

In [2]:
# Download model and effective area

eff_a = pd.read_csv('EffA_GC_approx.csv').values
model_1 = np.loadtxt("second_GC_model.txt", comments="#", unpack=False)

In [20]:
num_nu = int(model_1.size/30)

positions = np.column_stack((model_1[:, 23], model_1[:, 24], model_1[:, 25]))

bin_size=0.3
num_xbins, num_ybins, num_zbins = int(60/bin_size), int(60/bin_size), int(30/bin_size)
H, edges = np.histogramdd(positions, bins=(num_xbins, num_ybins, num_zbins), range=[(-30, 30), (-30, 30), (-3, 3)])

prob_dens_dist = np.divide(H, num_nu*(bin_size**2)*(5*bin_size))

x = np.linspace(-30, 30, num_xbins)
y = np.linspace(-30, 30, num_ybins)
z = np.linspace(-3, 3, num_zbins)
interp = RegularGridInterpolator((x, y, z), prob_dens_dist, method='linear', bounds_error=False, fill_value=0)

In [13]:
# Some core function definitions:

# l_eff per particle, in kpc:

def Leff(x, y, z):
    return np.sqrt((8-x)**2 + y**2 + z**2)

# Oscillation probability: Prob of oscillating into an electron neutrino (still assuming 1:2:0 flavor dist at source)
# Energy in TeV

def OscProb(dm2s, e_true, l_eff):

    l_eff = l_eff*conv_factor

    osc1 = (np.cos((dm2s[0] * l_eff)/(4*e_true*1e12)))**2
    osc2 = (np.cos((dm2s[1] * l_eff)/(4*e_true*1e12)))**2
    osc3 = (np.cos((dm2s[2] * l_eff)/(4*e_true*1e12)))**2

    prob_surv = osc1*(u_e[1]*u_e[1]) + osc2*(u_e[2]*u_e[2]) + osc3*(u_e[3]*u_e[3])
    prob_mue = osc1*(u_e[1]*u_m[1]) + osc2*(u_e[2]*u_m[2]) + osc3*(u_e[3]*u_m[3])
    prob_e = 0.666666*prob_mue + 0.333333*prob_surv 

    return prob_e


# Effective Area [cm^2]

x, y = [], []
for i in range(len(eff_a)):
    x.append(eff_a[i][0])
    y.append(eff_a[i][1])
f = interp1d(x, y, bounds_error=False, fill_value="extrapolate")
# f = interp1d(x, y)
# f = interp1d(x, y, bounds_error=False, fill_value=0)

def EffA(e_true):
# if (e_true >= eff_a[0][0] and e_true <= eff_a[len(eff_a)-1][0]):
    effa = 10000*f(e_true)
    return effa


# Flux function [kpc^-3]:

def Flux(e_true, x, y, z):
    return (interp([x, y, z]))*phi0*(e_true/e0)**(-index)

flux = np.vectorize(Flux, excluded=['x', 'y', 'z'])
vecflux = np.vectorize(Flux)

#Galactic coords to (x, y, z) centered at the galactic center

def Cartesian(r, l, b):
    # r, l, b = coords[0], coords[1], coords[2]
    x = 8 - r*np.cos(l)*np.cos(b)
    y = r*np.sin(l)*np.cos(b)
    z = r*np.sin(b)
    return [x, y, z]

def Cartx(r, l, b):
    x = 8 - r*np.cos(l)*np.cos(b)
    return x 

def Carty(r, l, b):
    y = r*np.sin(l)*np.cos(b)
    return y

def Cartz(r, l, b):
    z = r*np.sin(b)
    return z

0.004187800796100576